In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("HW_1_data_numpy.ipynb")

# HW 1: Statistical analysis of data using numpy and matplotlib

Resources: Lecture slides describing the homework: https://docs.google.com/presentation/d/1ef0msC9XIT37_Yg94Cf4TL9VBgbIIfkKik8jgshskjE/edit?usp=sharing

Please do lab 1 before starting this homework, and do lab 2 before tackling problem 4 on (the plotting part).

In this homework the focus is on code *design*. Most of the functionality of the code will be what you did in the labs. I don't expect you to do the code design; I've provided that. Just pay attention to how a couple of tools (functions, dictionaries) can make your code a little more re-usable, cleaner, and less prone to error. 

Learning to *read* code is also as important as *writing* code. This is (hopefully) a gentle introduction to using/interacting with more advanced concepts/syntax/semantics. 

Where you're going with Problems 1, 2, and 3: Take a look in the **Data** directory at the **week_1_check_results.json** file. This is what your code should produce after those problems. 

## Setting up libraries and reading in data
Import the libraries that you will need (numpy, json, matplotlib for week 2) and read in the data you will need.

_Type your answer here, replacing this text._

In [ ]:
# Libraries that we need to import - numpy and json (for loading the description file)

# Need this for autograding to work
import otter
grader = otter.Notebook()


# TODO: put the numpy and json imports here so you can use those libraries (see the top of Lab 1 & 2)

In [ ]:
pick_data = ...
# Code to read in pick_data_description
pick_data_description = ...
# Code to get data_channels out of pick_data_description
data_channels = ...

In [ ]:
pick_data_description["n_total_dims"] = ...
pick_data_description["n_time_steps"] = ...

In [ ]:
grader.check("initialize_data")

##  Print stats for each sensor channel

This is almost entirely copying your code from lab 1 into a new structure that makes it easier to generalize your code. I've provided the structure, and broken the process up into 3 steps.

- Generalization 1: Put the code that calculates the statistics into a function
- Generalization 2: Store the results in a dictionary so it's labeled and you don't have to worry about variable name re-use

Note: You should not use **pick_data** and **pick_data_description** *inside* your function. They are passed in as variables to the function.

I've set this up so that you can write the function, then check it, then incrementally add in the **for** loops. You're free to jump straight to step 3 if you want, but if it doesn't work, please go through steps 1 and 2 before asking the TA for help.

_Type your answer here, replacing this text._

In [ ]:
def get_stats_for_channel(data, channel_info, xyz_dim, n_total_dims):
    """ Get the min, max, mean, sd for the given channel
    @param data - the numpy array from the csv file (pick_data)
    @param channel_info - a dictionary with the channel info, which has in it the name, index_offset, and dimensions
    @param xyz_dim - 0, 1, or 2 for x, y, z
    @param n_total_dims - total number of columns per one time step
    @returns - A dictionary with the statistics values """

    # This is index_wrist_torque_offset from lab 1 - the start index from the dictionary plus the dimension (xyz_dim)
    # Do the same thing you did in lab 1 (get the index_offset from channel_info) then add in the x,y,z offset
    index_offset = ... 
    
    # The np.min(data[:, start:end:stop]) code from lab replaces the 0 
    ret_stats = {"Min": 0,
                 "Max": 0,
                 "Mean": 0,
                 "SD": 0}
    return ret_stats


In [ ]:
ret_stats = get_stats_for_channel(pick_data, data_channels[0], 0, pick_data_description["n_total_dims"])
print(ret_stats)

In [ ]:

#. for each item in data channels
#.    Print the channel name
#.    Get the stats m_stats = get_...
#     print(f"  minimum: {my_stats['Min']}, maximum: {my_stats['Max']}, mean: {my_stats['Mean']}, SD: {my_stats['SD']}")

In [ ]:
# Cute trick to map 0-2 to letters
map_to_xyz = ['x', 'y', 'z']

#. for each item in data channels
#.    Print the channel name
#.    Create an empty list d["stats"] = [] 
#.    for each dimension in channel
#.       Get the stats m_stats = get_...
#.       ... and store them in d["stats"]
#        print(f"  minimum: {my_stats['Min']}, maximum: {my_stats['Max']}, mean: {my_stats['Mean']}, SD: {my_stats['SD']}")

In [ ]:
# Write the results back out to a text file you can read in a text editor
with open('Data/week1_student_results.json', 'w') as f:
    json.dump(pick_data_description, f, indent=4)

In [ ]:
grader.check("calc_stats_in_function")

## Max peak per channel

Find the row(s) with the maximum peak in the Wrist torque Z channel

This is the optional problem from lab 1.

Notes for problem 3
 - No for loops - do this with np.where.
 - We should be able to change channel_to_search to a different text string and it still works.
 - i.e., no "hard-wiring" the channel name/index

Optional: Do this for all channels and dimensions

Recommended order of implementation for the optional version (see lab 1 for the non-optional version):
 - Write the code for one channel, make sure it works
 - Create a function just like the one above (it will take the same input parameters)
 --    The return value will be different - I suggest a list of tuples with [(r, c), (r, c)]
 -  Copy your code for one channel into the function, and change it to take in the function's input parameters
 - For the output, I suggest making an empty list, and then, in the for loop, append all valid r,c pairs onto the list. Return the list.
 - Copy your for loop from above (that goes over all channels, all dimensions) and replace the function call with the new one.

In [ ]:

# This should be of the form (r,c)
row_max_wrist_torque_z = ...

In [ ]:
grader.check("channel_index")

## Now use slicing to get out all of the wrist Force data and calculate the min and the max

This is a pretty complicated slice. Steps to get there:
-First, use the slice operator, selecting all rows and columns, **data[:, :]** to calculate the minimum across the entire matrix of data. This should be the same as **np.min(pick_data)**
- Now change the column slice from all columns to starting at the offset value.
- Now change the slice to take a step/stride of **n_total_dims** instead of 1
- Reminder: slicing is  **start:end:step**

Note: You don't need to put an end value in - just leave it blank

Remember: The data is in **pick_data**, not **pick_data_description**

In [ ]:

min_wrist_force = ...
max_wrist_force = ...
print(f"Minimum {min_wrist_force} and maximum {max_wrist_force} value of wrist force z channel")


In [ ]:
grader.check("slicing")

## Boolean slicing to get successful versus unsuccessful picks out

Now for the fun one - do the same but ONLY for successful (last column is 1) picks versus unsuccessful (last column is 0)

Note: Use == 1, not **is** 1

It may be helpful here to explicitly create a boolean array that is # rows X 1 to use as the row index for the
pick_data, rather than trying to do it "in-place". Then you can check that the array is correct (print it out - it should have Trues and Falses in the same order as the 1s and 0s in the spreadsheet's last column).

This boolean array replaces the **:** row index: **data[boolean_array, start:end:step]**

The boolean array is created by doing a comparison, for example, **data == 1**. In this case, you want to get out all rows, but only the LAST column of the data, and compare it to 1.

In [ ]:
min_wrist_force_successful = ...
max_wrist_force_successful = ...

min_wrist_force_unsuccessful = ...
max_wrist_force_unsuccessful = ...

print(f"Successful: Minimum {min_wrist_force_successful} and maximum {max_wrist_force_successful} value of wrist force-torque chanel")
print(f"Unsuccessful: Minimum {min_wrist_force_unsuccessful} and maximum {max_wrist_force_unsuccessful} value of wrist force-torque chanel")

In [ ]:
grader.check("boolean_slicing")

<!-- BEGIN QUESTION -->

## Optional: print out all of the indices where the maximum value for the successful pick was reached

np.where is the method you want; it returns a tuple (think list) with two lists, one for each dimension. To get out the pairs of indices, you want the first element of the first list and the first element of the second list, and so on. You can do this with a for loop, using zip to zip together the two arrays.

- for r, c in zip( ):

Some gotchas: it's easiest to do where on the entire pick_data set, but then you'll get indices that are NOT the
ones you want (other data channels). If you do max on the sliced matrix, then you'll get indices on the sliced
matrix, not the original... So either you need to filter out indices that are not the channel you want with an
**if** statement (the modulo operator **%** is useful here) OR adjust the column index by doing offset + index * n_total_dims.

See the slides for help on converting to and fro from row column indexing, and see the Miro diagram for more info on re-configuring the output of **where**.

This is OPTIONAL for the lab, but we will be coming back to this in the homework. Extra credit grades are adjusted in Canvas, btw.

In [ ]:
# Use where to get out the indices. You can use == OR np.isclose() here; either works, but np.isclose will give 
#. you slightly more results

all_indices_from_where = ...
# for all row, column in all_indices_from_where
#.   if this is the column for wrist force 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")


<!-- END QUESTION -->



---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit just the .ipynb file to Gradescope. Don't change the provided variable names or autograding will fail.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()